# Name: Jawad Hassan
# Section: Bs Ai
# Roll No: 2230-0035
# Teacher: Ms Salma Asif

In [10]:
#widgets
!pip install ipycanvas ipywidgets
from google.colab import output
output.enable_custom_widget_manager()


In [12]:
#imports
from ipycanvas import Canvas, MultiCanvas
from ipywidgets import Button, HBox, VBox, Label, Output
import random
import time


In [14]:
#setup
CELL = 60
BOARD_SIZE = 10
SIZE = CELL * BOARD_SIZE

# Multi-layer canvas (layer 0 = board, layer 1 = tokens)
multicanvas = MultiCanvas(2, width=SIZE, height=SIZE)
canvas = multicanvas[0]        # board
token_canvas = multicanvas[1]  # tokens


In [16]:
#pixel coordinates
def square_to_xy(square):
    n = square - 1
    row = n // BOARD_SIZE
    col = n % BOARD_SIZE

    # Zigzag numbering
    if row % 2 == 1:
        col = BOARD_SIZE - 1 - col

    x = col * CELL + CELL / 2
    y = (BOARD_SIZE - 1 - row) * CELL + CELL / 2
    return x, y


In [17]:
#board grid
def draw_board():
    canvas.clear()

    # Grid lines
    for i in range(BOARD_SIZE + 1):
        canvas.stroke_line(0, i * CELL, SIZE, i * CELL)
        canvas.stroke_line(i * CELL, 0, i * CELL, SIZE)

    # Numbers
    for s in range(1, 101):
        x, y = square_to_xy(s)
        canvas.fill_text(str(s), x - 15, y - 15)

draw_board()
multicanvas


MultiCanvas(height=600, width=600)

In [18]:
snakes = {16:6, 47:26, 49:11, 56:53, 62:19, 64:60, 87:24, 93:73, 95:75, 98:78}
ladders = {1:38, 4:14, 9:31, 21:42, 28:84, 36:44, 51:67, 71:91, 80:100}

def draw_snakes_ladders():
    # snakes in red
    canvas.stroke_style = "red"
    for a, b in snakes.items():
        x1, y1 = square_to_xy(a)
        x2, y2 = square_to_xy(b)
        canvas.stroke_line(x1, y1, x2, y2)

    # ladders in green
    canvas.stroke_style = "green"
    for a, b in ladders.items():
        x1, y1 = square_to_xy(a)
        x2, y2 = square_to_xy(b)
        canvas.stroke_line(x1, y1, x2, y2)

draw_board()
draw_snakes_ladders()
multicanvas


MultiCanvas(height=600, width=600)

In [19]:
players = [
    {"pos": 1, "color": "blue"},
    {"pos": 1, "color": "red"}
]

current_player = 0

def draw_tokens():
    token_canvas.clear()
    for i, p in enumerate(players):
        x, y = square_to_xy(p["pos"])
        offset = -10 if i == 0 else 10
        token_canvas.fill_style = p["color"]
        token_canvas.fill_circle(x, y + offset, 12)


In [20]:
def animate_move(player_idx, start, end):
    steps = 15
    x1, y1 = square_to_xy(start)
    x2, y2 = square_to_xy(end)

    for t in range(steps + 1):
        # linear interpolation
        x = x1 + (x2 - x1) * (t / steps)
        y = y1 + (y2 - y1) * (t / steps)

        token_canvas.clear()
        for i, p in enumerate(players):
            if i == player_idx:
                offs = -10 if i == 0 else 10
                token_canvas.fill_style = p["color"]
                token_canvas.fill_circle(x, y + offs, 12)
            else:
                px, py = square_to_xy(p["pos"])
                offs = -10 if i == 0 else 10
                token_canvas.fill_style = p["color"]
                token_canvas.fill_circle(px, py + offs, 12)

        time.sleep(0.02)


In [21]:
status = Label(value="Player 1's turn")
out = Output()

def roll_dice(_):
    global current_player
    p = players[current_player]

    roll = random.randint(1, 6)
    with out:
        out.clear_output()
        print(f"Player {current_player+1} rolled:", roll)

    new_pos = p["pos"] + roll
    if new_pos > 100:
        new_pos = p["pos"]  # cannot overshoot final square

    # animate step-by-step movement
    for sq in range(p["pos"] + 1, new_pos + 1):
        animate_move(current_player, p["pos"], sq)
        p["pos"] = sq

    # snake or ladder jump
    if new_pos in ladders:
        dest = ladders[new_pos]
        with out:
            print("Ladder! →", dest)
        animate_move(current_player, new_pos, dest)
        p["pos"] = dest

    elif new_pos in snakes:
        dest = snakes[new_pos]
        with out:
            print("Snake! →", dest)
        animate_move(current_player, new_pos, dest)
        p["pos"] = dest

    draw_tokens()

    # win check
    if p["pos"] == 100:
        status.value = f"Player {current_player+1} wins! 🎉"
        roll_btn.disabled = True
        return

    # next turn
    current_player = 1 - current_player
    status.value = f"Player {current_player+1}'s turn"

roll_btn = Button(description="🎲 Roll Dice")
roll_btn.on_click(roll_dice)

ui = VBox([multicanvas, HBox([roll_btn, status]), out])
draw_tokens()
ui
